In [21]:
import numpy as np
import torch
from utils.motion import *
from utils.skeleton import Skeleton

In [22]:
data = np.load("/source/junhyuk/motion-style/results/4/reconstruct/030075_reconstructed.npy")
data = torch.from_numpy(data).float()

In [23]:
skel = np.load("skeleton.npz", allow_pickle=True)
offset = torch.tensor(skel["offsets"], dtype=torch.float32)
kinematic_tree = [list(chain) for chain in skel["kinematic_tree"]]

In [24]:
skeleton = Skeleton(offset, kinematic_tree, device='cpu')
skeleton.set_offset(offset)

In [25]:
# r_rot_quat, r_pos = recover_root_rot_pos(data)
joint_pos = recover_from_rot(data, 22, skeleton).numpy()

In [28]:
kinematic_chain = [[0, 2, 5, 8, 11], [0, 1, 4, 7, 10], [0, 3, 6, 9, 12, 15], [9, 14, 17, 19, 21], [9, 13, 16, 18, 20]]

In [ ]:
# motion_seq = torch.from_numpy(np.load("/source/junhyuk/motion-style/results/3/reconstruct/030075_reconstructed.npy")).float()
# joint = recover_from_ric(motion_seq, 22).numpy()

In [30]:
import os
from salad.utils.plot_script import plot_3d_motion
save_path = os.path.join("./delete.mp4")
plot_3d_motion(save_path, kinematic_chain, joint_pos, title='test', fps=20);

/source/junhyuk/motion-style/salad/utils/plot_script.py:414: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "dpi" which is no longer supported as of 3.3 and will become an error two minor releases later
  ani.save(save_path, fps=fps)
/source/junhyuk/motion-style/salad/utils/plot_script.py:414: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "facecolor" which is no longer supported as of 3.3 and will become an error two minor releases later
  ani.save(save_path, fps=fps)
/source/junhyuk/motion-style/salad/utils/plot_script.py:414: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "edgecolor" which is no longer supported as of 3.3 and will become an error two minor releases later
  ani.save(save_path, fps=fps)
/source/junhyuk/motion-style/salad/utils/plot_script.py:414: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "orientation" which is no longer supported as of 3.3 and will become an error 

In [ ]:
from glob import glob

device = 'cuda:0'
opt = get_opt("checkpoints/t2m/t2m_denoiser_vpred_vaegelu/opt.txt", device)

input_folder = "/source/junhyuk/motion-style/results/4/style"
output_folder = "video/4/style"
os.makedirs(output_folder, exist_ok=True)

npy_files = sorted(glob(os.path.join(input_folder, "*.npy")))

for npy_file in npy_files:
    filename = os.path.basename(npy_file)
    basename = os.path.splitext(filename)[0]  # e.g., "030075_reconstructed"

    data = np.load(npy_file)
    joint = recover_from_ric(torch.from_numpy(data).float(), opt.joints_num).numpy()

    save_path = os.path.join(output_folder, f"{basename}.mp4")
    plot_3d_motion(save_path, opt.kinematic_chain, joint, title=basename, fps=20)

    print("✅ Saved", save_path)

In [35]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def show_frame_by_frame(joint_seq, skeleton):
    for t in range(joint_seq.shape[0]):
        joints = joint_seq[t]

        fig = plt.figure(figsize=(6, 6))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_title(f"Frame {t}")
        ax.set_xlim(-1, 1)
        ax.set_ylim(-1, 1)
        ax.set_zlim(0, 2)
        ax.set_xlabel("X")
        ax.set_ylabel("Z")
        ax.set_zlabel("Y")
        ax.view_init(elev=110, azim=-90)

        # Draw joints and bones
        ax.scatter(joints[:, 0], joints[:, 2], joints[:, 1], c='blue', s=20)
        for i, j in skeleton:
            ax.plot(
                [joints[i, 0], joints[j, 0]],
                [joints[i, 2], joints[j, 2]],
                [joints[i, 1], joints[j, 1]],
                c='blue', linewidth=2
            )

        plt.show()
        input("Press Enter for next frame...")
        plt.close(fig)